<a href="https://colab.research.google.com/github/Jasper-Hewitt/Wind_project/blob/main/alternative_finding_noisepollution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#this one uses a different regex, split on 。 and ：
#many of the units of information within the PDF are actually only one sentence so I guess this is fine. 
!pip install fitz
!pip install static
!pip install frontend
!pip install pymupdf --upgrade

In [ ]:
# mkdir static
import fitz

doc = fitz.open("dragged_130_146.pdf")

for page in doc:
    text = page.get_text("text")
    with open("output3.txt", "a") as f:
        f.write(text)


In [ ]:
with open('output3.txt', 'r') as file:
    # Read the contents of the file
    contents = file.read()
    # Print the contents
    print(contents)

In [ ]:
import re
#pattern = "(\(一\))|(\(二\))|(\(三\))|(\(四\))|(\(五\))|(\(六\))|(\(七\))|(\(八\))|(\(九\))|(\(十\))|(^[0-9]+\.)"
#split_contents = re.split(pattern, contents)

In [ ]:
pattern = "。|:"

split_contents = re.split(pattern, contents)

# print(sentences)


In [ ]:
split_contents

In [ ]:
len(split_contents)

128

In [ ]:
# import re
# split_contents = re.split(r'\d+\.', contents)
# split_contents

In [ ]:
!pip install sentence_transformers
#also works with entire alineas 
#below are some other models that don't work. I get a warning signal that says it can't find the model so it switches to MEAN pooling instead. 
#('all-MiniLM-L6-v2'), one of the most popular ones, works but it is an English model 
#("uer/sbert-base-chinese-nli")#('shibing624/text2vec-base-chinese')#('all-MiniLM-L6-v2') 

from sentence_transformers import SentenceTransformer, util
import torch

#alternative model by same creators: ("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"). This one appears to be more popular. Compare results.
embedder = SentenceTransformer("distiluse-base-multilingual-cased-v1")

# Corpus with example sentences
corpus = split_contents
corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)

# Query sentences:
queries = ['噪音', '水下噪音','聲音']

# Find the closest X sentences of the corpus for each query sentence based on cosine similarity
top_k = min(68, len(corpus))
for query in queries:
    query_embedding = embedder.encode(query, convert_to_tensor=True)

    # We use cosine-similarity and torch.topk to find the highest 5 scores
    cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop X most similar sentences in corpus:")

    for score, idx in zip(top_results[0], top_results[1]):
        print(corpus[idx], "(Score: {:.4f})".format(score))

#calculate accumulated similarity score per keyword

In [ ]:
sentence_scores = {sentence: {query: 0 for query in queries} for sentence in corpus}

for query in queries:
    query_embedding = embedder.encode(query, convert_to_tensor=True)
    cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=len(corpus))
    for score, idx in zip(top_results[0], top_results[1]):
        sentence = corpus[idx]
        sentence_scores[sentence][query] = score

In [ ]:
len(sentence_scores)

94

In [ ]:
import pandas as pd
#in case we use more or less keywords, we will have to adjust this code slightly
df = pd.DataFrame([[sentence, scores[queries[0]], scores[queries[1]], 
                    scores[queries[2]], scores[queries[0]]+scores[queries[1]]+scores[queries[2]]] 
                   for sentence, scores in sentence_scores.items()], 
                  columns=['text', 'similarity_score1', 'similarity_score2', 'similarity_score3', 
                           'accumulated_score'])
df = df.sort_values(by='accumulated_score', ascending=False)

In [ ]:
#output
df=df.reset_index(drop=True)
df

,text,similarity_score1,similarity_score2,similarity_score3,accumulated_score
0,\n二、 水下噪音監測 \n水下噪音監測之目的係為確認打樁期\n間之水下噪音值是否超出�值...,tensor(0.3657),tensor(0.5341),tensor(0.2998),tensor(1.1996)
1,\n��,tensor(0.4494),tensor(0.2770),tensor(0.4720),tensor(1.1983)
2,\n－ \n－ \n(2)依據噪音振動模擬\n結果顯示，陸上施\n工及風機營運後之\n全頻...,tensor(0.4357),tensor(0.3761),tensor(0.3688),tensor(1.1805)
3,\n3.開發行為屬點狀開\n發 ， 無 大 面 積 施\n工，環境影響說明\n書中已針對施...,tensor(0.2976),tensor(0.3847),tensor(0.2543),tensor(0.9367)
4,\n8.1.1 \n8-1 \n(二)施工期間於距離打樁\n位置外750公尺處選擇\n合理...,tensor(0.2711),tensor(0.3769),tensor(0.2498),tensor(0.8978)
...,...,...,...,...,...
89,\n回饋計畫」有關內容納入附錄十八，說明\n如下： \n目前本計畫已與彰化縣政府達成初步共...,tensor(-0.0404),tensor(-0.0144),tensor(-0.0151),tensor(-0.0698)
90,\n(二) 黃金標準 (Gold Standard)：黃金\n標準為由世界自然基金會和其他...,tensor(-0.0172),tensor(-0.0200),tensor(-0.0453),tensor(-0.0825)
91,\n二、 在地���花�原則上也會配合辦理\n,tensor(-0.0652),tensor(0.0127),tensor(-0.0422),tensor(-0.0947)
92,\n(五) 邀請丹麥頂尖大學來台在國內及\n彰化地區大學開課,tensor(-0.0552),tensor(-0.0498),tensor(-0.0042),tensor(-0.1092)
